In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe2+ intake
                                        if 'EX_fe2_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2,  8,  5,  3,  2, 10, 14, 16,  4,  7,  7,  5, 13, 18, 11, 12,  9,
        6, 15, 18, 13,  1,  9, 18,  2,  6, 17,  6, 13, 17]), 1: array([ 3, 16,  3, 11,  7, 13, 12,  9,  3, 17, 19,  9,  2,  9,  9,  8,  2,
       13, 20, 20,  9, 15, 17,  8, 13, 16,  7,  2,  9,  8])}
1
strain_number:  0 11.066666666666666
strain_various:  0 6.428754847471545
strain_number:  1 10.3
strain_various:  1 5.4169487106057534
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2,  8,  5,  3,  2, 10, 14, 16,  4,  7,  7,  5, 13, 18, 11, 12,  9,
        6, 15, 18, 13,  1,  9, 18,  2,  6, 17,  6, 13, 17]), 1: array([ 3, 16,  3, 11,  7, 13, 12,  9,  3, 17, 19,  9,  2,  9,  9,  8,  2,
       13, 20, 20,  9, 15, 17,  8, 13, 16,  7,  2,  9,  8])}
_2:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862

glc__D_e_1:  -19.309320310904127
glc__D_e_1:  -18.405302157591457
glc__D_e_1:  -28.69382104207201
glc__D_e_1:  -19.010979117375225
glc__D_e_1:  -19.778010257495296
glc__D_e_1:  -0.7543535379206465
glc__D_e_1:  -18.98306491461538
glc__D_e_1:  -26.997709376546666
glc__D_e_1:  -0.17657828925836938
glc__D_e_1:  -18.156828885053013
glc__D_e_1:  -26.218714558340356
glc__D_e_1:  -0.06625751948456582
glc__D_e_1:  -0.3055197179014345
glc__D_e_1:  -12.734905733877897
glc__D_e_1:  -16.686200431569095
glc__D_e_1:  -1.2242231677158433
glc__D_e_1:  -2.2008928224156534
glc__D_e_1:  -38.35479167769414
glc__D_e_1:  -17.646626119863658
glc__D_e_1:  -0.6574374595814731
glc__D_e_1:  -0.4372529047111945
glc__D_e_1:  -27.10134662426887
glc__D_e_1:  -17.850303148298103
glc__D_e_1:  -0.1924404395834074
glc__D_e_1:  -0.5250593773182768
glc__D_e_1:  -34.0429012504521
glc__D_e_1:  -8.259634960917058
glc__D_e_1:  -29.22581857478145
glc__D_e_1:  -13.200238280967461
glc__D_e_1:  -0.3114439383998413
glc__D_e_1:  -20

glc__D_e_1:  -12.515441747482434
glc__D_e_1:  -8.240253042044044
glc__D_e_1:  -0.7300592788321898
glc__D_e_1:  -4.4950712035038665
glc__D_e_1:  -0.3972217730479306
glc__D_e_1:  -1.8469945308394105
glc__D_e_1:  -0.5080775921533547
glc__D_e_1:  -0.3282417114031172
glc__D_e_1:  -0.37110256248090345
glc__D_e_1:  -0.41859220208176473
glc__D_e_1:  -0.6670589937393584
glc__D_e_1:  -0.07042881091092834
glc__D_e_1:  -0.18003764909229303
glc__D_e_1:  -0.67476053007358
glc__D_e_1:  -0.5580886642262587
glc__D_e_1:  -0.24344501485010217
glc__D_e_1:  -0.7382128510700161
glc__D_e_1:  -0.7854022436980639
strain_number:  0 10.166666666666666
strain_various:  0 19.606263171638687
strain_number:  1 0.03333333333333333
strain_various:  1 0.1795054935711501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 711.6655934364911, 337.1172070318742, 382.7229316735895, 445.05924338819074, 416.7996435581963, 481.0154746569221, 479.16359103411315, 404.10916145355213, 354.1447329369674, 348.25923124564434, 335.713597869892

glc__D_e_1:  -1.764263748961346
glc__D_e_1:  -1.8324669960823214
glc__D_e_1:  -0.42042371671144807
strain_number:  0 0.23333333333333334
strain_various:  0 0.7608474807008886
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 711.6655934364911, 337.1172070318742, 382.7229316735895, 445.05924338819074, 416.7996435581963, 481.0154746569221, 479.16359103411315, 404.10916145355213, 354.1447329369674, 348.25923124564434, 335.71359786989257, 339.09467302864493, 353.57916751465075, 383.57821244242166, 276.5537684209525, 242.20973726661416, 197.98034171342442, 192.12589294090847, 189.8105231985529, 204.9997401690925, 174.46062243677375, 247.17024379721303, 83.60574770057528, 70.14227086183311, 56.34189723320158, 38.51809954751131, 26.166666666666668, 25.480769230769234, 29.57142857142857]
slip_r:  29.57142857142857
31
glc__D_e_1:  -0.2919298337044196
strain_number:  0 0.13333333333333333
strain_various:  0 0.42687494916218993
strain_number:  1 0.0
strain_va

In [7]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10,  8, 19, 19,  2,  7,  1, 17, 16, 15,  3, 19, 12, 15,  3, 11, 10,
       14, 13, 11,  5, 15, 13,  6, 14,  5,  1, 13, 18, 13]), 1: array([ 2,  3, 12,  8,  7,  9, 20,  7, 17, 15, 19,  4, 11,  5, 19,  2, 13,
        8,  5, 13, 12, 13,  1, 11, 11, 17, 13,  3, 14, 10])}
1
strain_number:  0 12.733333333333333
strain_various:  0 6.546924638501823
strain_number:  1 10.133333333333333
strain_various:  1 5.377318621353542
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10,  8, 19, 19,  2,  7,  1, 17, 16, 15,  3, 19, 12, 15,  3, 11, 10,
       14, 13, 11,  5, 15, 13,  6, 14,  5,  1, 13, 18, 13]), 1: array([ 2,  3, 12,  8,  7,  9, 20,  7, 17, 15, 19,  4, 11,  5, 19,  2, 13,
        8,  5, 13, 12, 13,  1, 11, 11, 17, 13,  3, 14, 10])}
_2:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0.2857391744693639], 

glc__D_e_1:  -3.3668413933931873
glc__D_e_1:  -16.82121095664575
glc__D_e_1:  -0.4284181657988011
glc__D_e_1:  -9.344142965150684
glc__D_e_1:  -14.607517520791454
glc__D_e_1:  -27.099688710661162
glc__D_e_1:  -10.27956191217694
glc__D_e_1:  -0.21559391452516952
glc__D_e_1:  -25.54505135431638
glc__D_e_1:  -23.389421709863637
glc__D_e_1:  -0.1950165585360164
glc__D_e_1:  -21.54373089924775
strain_number:  0 43.03333333333333
strain_various:  0 13.1693668117424
strain_number:  1 1.1
strain_various:  1 0.9433981132056605
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 934.2310336722102, 394.11819308142833, 393.1472436876849, 452.49819916925185, 560.5203265602647, 548.0955363264187, 425.1385062606812, 431.5863971489377, 404.8858986302557, 422.78196229718833, 398.45869907196436, 308.3021399519397, 350.2726154695174]
slip_r:  350.2726154695174
15
glc__D_e_1:  -27.056898197625188
glc__D_e_1:  -19.146617566184176
glc__D_e_1:  -36.150964894721895
glc__D_e_1:  -25.29215102645231
glc__D_e_1:  -29.8148

glc__D_e_1:  -1.3119931138245218
glc__D_e_1:  -2.0212878930996627
glc__D_e_1:  -1.5471432964383685
glc__D_e_1:  -0.280675955504903
glc__D_e_1:  -0.7509585188292862
glc__D_e_1:  -0.20690899826478715
glc__D_e_1:  -0.9712479471132704
glc__D_e_1:  -1.7493085156924333
glc__D_e_1:  -0.7115110461299974
glc__D_e_1:  -0.2685876742351363
glc__D_e_1:  -1.1858423761835089
glc__D_e_1:  -0.8630508003550474
glc__D_e_1:  -1.199424056946414
glc__D_e_1:  -1.130228540050354
glc__D_e_1:  -1.2047414823712748
glc__D_e_1:  -0.02217352167550324
glc__D_e_1:  -1.1368801097094372
glc__D_e_1:  -1.0078223188418292
glc__D_e_1:  -1.2468719284465335
glc__D_e_1:  -1.9592370018057887
glc__D_e_1:  -2.896603339637201
glc__D_e_1:  -0.8550373000464248
glc__D_e_1:  -2.5129166895357455
glc__D_e_1:  -0.09544679481282592
glc__D_e_1:  -2.3122829982085715
glc__D_e_1:  -0.684592266995459
glc__D_e_1:  -0.9802748272928594
glc__D_e_1:  -0.5165298456949803
glc__D_e_1:  -0.20139982414225988
strain_number:  0 2.433333333333333
strain_v

In [8]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7, 13, 15, 17, 14, 18,  9,  8,  7, 18, 14, 14, 11, 17, 12, 13,  9,
       16, 17,  8, 16, 17, 11, 14, 17, 16, 11, 10, 13, 11]), 1: array([11,  2,  2,  2, 12, 14, 18,  8,  1, 19, 12,  1, 10, 20, 20,  5,  6,
        5, 16,  9,  8,  8, 18,  6,  1, 17, 11,  8, 11, 13])}
1
strain_number:  0 15.266666666666667
strain_various:  0 4.090096440047459
strain_number:  1 9.8
strain_various:  1 5.95203046587185
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7, 13, 15, 17, 14, 18,  9,  8,  7, 18, 14, 14, 11, 17, 12, 13,  9,
       16, 17,  8, 16, 17, 11, 14, 17, 16, 11, 10, 13, 11]), 1: array([11,  2,  2,  2, 12, 14, 18,  8,  1, 19, 12,  1, 10, 20, 20,  5,  6,
        5, 16,  9,  8,  8, 18,  6,  1, 17, 11,  8, 11, 13])}
_2:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.520733775712074

glc__D_e_1:  -0.03478270599795508
glc__D_e_1:  -0.4878660709931043
glc__D_e_1:  -16.38064349748926
glc__D_e_1:  -47.1276780520634
glc__D_e_1:  -1.1280103077283954
glc__D_e_1:  -0.971639775133899
glc__D_e_1:  -14.259047836307392
glc__D_e_1:  -33.78211820174923
glc__D_e_1:  -2.140789363427526
glc__D_e_1:  -1.2019611884312393
glc__D_e_1:  -20.742255726379124
glc__D_e_1:  -43.99757307679081
glc__D_e_1:  -3.2298010000819097
glc__D_e_1:  -0.5429568713648854
glc__D_e_1:  -20.083251409312773
strain_number:  0 43.93333333333333
strain_various:  0 15.9037382048652
strain_number:  1 1.5333333333333334
strain_various:  1 1.2578641509408806
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 623.9589057098732, 370.15750915750914, 383.36079915415644, 446.5399965837786, 442.93155212513614, 386.28939521488775, 380.67425337293116, 422.4910669788821, 391.1549889060376, 358.0565730487472, 378.4436407397079, 343.91468138980304]
slip_r:  343.91468138980304
14
glc__D_e_1:  -31.568286066813116
glc__D_e_1:  -17.612927

glc__D_e_1:  -3.8920846254683443
glc__D_e_1:  -1.4516341945011766
glc__D_e_1:  -1.4614926372495545
glc__D_e_1:  -1.049962123443616
glc__D_e_1:  -4.457817160333452
glc__D_e_1:  -2.5122255292840654
glc__D_e_1:  -0.4037219504827163
glc__D_e_1:  -4.147383331229635
glc__D_e_1:  -2.3063973247871803
glc__D_e_1:  -1.3981272558287012
glc__D_e_1:  -1.3969562827036883
glc__D_e_1:  -1.4635417987808486
glc__D_e_1:  -1.3789646593870128
glc__D_e_1:  -0.24757227221512146
glc__D_e_1:  -0.5129771480709222
glc__D_e_1:  -1.1412323550051109
glc__D_e_1:  -0.3400844373846692
glc__D_e_1:  -0.9485513378974104
glc__D_e_1:  -0.7851795135635864
glc__D_e_1:  -1.010836510842744
glc__D_e_1:  -0.9551633894873578
glc__D_e_1:  -1.4768207578688852
glc__D_e_1:  -0.6756900596745201
glc__D_e_1:  -2.4604140383487487
glc__D_e_1:  -0.8999245962168702
glc__D_e_1:  -1.1062084701109354
glc__D_e_1:  -0.17281821388286023
strain_number:  0 2.8
strain_various:  0 1.6206994374857624
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_t

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10,  1,  2, 17, 20, 13, 15, 11, 14,  6, 16, 16, 14,  8, 19, 10,  6,
       18,  7, 18, 17, 14, 15, 20,  4,  7,  5,  9, 10, 11]), 1: array([18, 16, 17,  2, 11, 15,  6,  5,  2,  2,  6,  2,  2,  8,  2, 10,  4,
        1, 12, 15, 19, 18,  4,  7, 11, 20, 19,  3,  3,  5])}
1
strain_number:  0 13.766666666666667
strain_various:  0 6.401475524352873
strain_number:  1 8.833333333333334
strain_various:  1 6.377477732005203
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10,  1,  2, 17, 20, 13, 15, 11, 14,  6, 16, 16, 14,  8, 19, 10,  6,
       18,  7, 18, 17, 14, 15, 20,  4,  7,  5,  9, 10, 11]), 1: array([18, 16, 17,  2, 11, 15,  6,  5,  2,  2,  6,  2,  2,  8,  2, 10,  4,
        1, 12, 15, 19, 18,  4,  7, 11, 20, 19,  3,  3,  5])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.285739174469363

glc__D_e_1:  -37.437466014408905
glc__D_e_1:  -23.832169954085412
glc__D_e_1:  -45.953242262347985
glc__D_e_1:  -28.110507615265895
glc__D_e_1:  -16.06757598070673
glc__D_e_1:  -31.408340564657056
glc__D_e_1:  -16.898867762834872
glc__D_e_1:  -19.893275114087242
glc__D_e_1:  -18.61204812970424
glc__D_e_1:  -0.18488896887121342
glc__D_e_1:  -7.508311355911789
glc__D_e_1:  -30.884556402143993
glc__D_e_1:  -0.2844010012506972
glc__D_e_1:  -9.90611292762462
glc__D_e_1:  -22.28080494029176
glc__D_e_1:  -6.453767553098864
strain_number:  0 41.63333333333333
strain_various:  0 16.51359709115155
strain_number:  1 0.9
strain_various:  1 1.4224392195567914
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 572.8327071535276, 369.04484769616346, 377.7454956550159, 394.15752015220716, 405.48231987310936, 410.02088056996547, 388.5448529128051, 408.56001088391434, 338.80310936759327, 309.975323811331, 258.20880735291445, 293.61051124596287, 339.6685671294234]
slip_r:  339.6685671294234
15
glc__D_e_1:  -25.6

glc__D_e_1:  -0.9367295846363508
glc__D_e_1:  -5.964747565239947
glc__D_e_1:  -1.2362727621371237
glc__D_e_1:  -0.7595456230915588
glc__D_e_1:  -1.3953696844228638
glc__D_e_1:  -0.41418483298969877
glc__D_e_1:  -0.9974634536067001
glc__D_e_1:  -1.7477638116137206
glc__D_e_1:  -1.1084273500962183
glc__D_e_1:  -0.17247539013082358
glc__D_e_1:  -0.660248395664977
glc__D_e_1:  -0.48635826344689403
glc__D_e_1:  -0.6472472424795451
glc__D_e_1:  -1.12496041284672
glc__D_e_1:  -1.2169911066095682
glc__D_e_1:  -0.1316847443833451
glc__D_e_1:  -1.2742505085118236
glc__D_e_1:  -0.8714939781477726
glc__D_e_1:  -2.5755470577484196
glc__D_e_1:  -4.987992776904066
glc__D_e_1:  -4.882287749163769
glc__D_e_1:  -2.780226934035047
strain_number:  0 2.6666666666666665
strain_various:  0 2.180723631172817
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 572.8327071535276, 369.04484769616346, 377.7454956550159, 394.15752015220716, 405.48231987310936, 410.02088056996547

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([18,  4, 13,  5, 12,  4, 15,  9,  2,  7,  6,  7, 17, 10,  3, 13,  3,
        2, 17, 13,  4, 17,  6,  8, 14, 13, 16, 15, 15,  4]), 1: array([16, 15,  6,  8, 17, 15,  5, 12,  8, 13,  7, 18, 17, 16, 18, 13, 12,
        6, 12, 18,  8,  7, 11,  1,  3, 18, 10, 12,  5, 20])}
1
strain_number:  0 11.233333333333333
strain_various:  0 6.370156111814599
strain_number:  1 11.566666666666666
strain_various:  1 5.096948978446703
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([18,  4, 13,  5, 12,  4, 15,  9,  2,  7,  6,  7, 17, 10,  3, 13,  3,
        2, 17, 13,  4, 17,  6,  8, 14, 13, 16, 15, 15,  4]), 1: array([16, 15,  6,  8, 17, 15,  5, 12,  8, 13,  7, 18, 17, 16, 18, 13, 12,
        6, 12, 18,  8,  7, 11,  1,  3, 18, 10, 12,  5, 20])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -25.363491169850537
glc__D_e_1:  -2.192293257360056
glc__D_e_1:  -1.6561200533299398
glc__D_e_1:  -17.52199252527129
glc__D_e_1:  -13.794431804174915
glc__D_e_1:  -26.880913397296162
glc__D_e_1:  -14.486952080143745
glc__D_e_1:  -1.289507104135085
glc__D_e_1:  -23.32263455593801
glc__D_e_1:  -11.581332077738107
glc__D_e_1:  -2.093254574423197
glc__D_e_1:  -12.843370033269007
glc__D_e_1:  -18.9373539104774
glc__D_e_1:  -25.76603971413936
glc__D_e_1:  -21.74220938476839
glc__D_e_1:  -17.809832920376596
glc__D_e_1:  -40.837773303789746
glc__D_e_1:  -14.340147290144271
glc__D_e_1:  -10.388685298224713
glc__D_e_1:  -19.669642593079317
glc__D_e_1:  -6.789549258180788
glc__D_e_1:  -18.948782988207036
glc__D_e_1:  -0.5216238273740075
glc__D_e_1:  -1.2240592297269854
glc__D_e_1:  -4.601610610533479
strain_number:  0 38.233333333333334
strain_various:  0 15.694337266104046
strain_number:  1 2.1666666666666665
strain_various:  1 1.8633899812498247
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1

glc__D_e_1:  -3.21997964907157
glc__D_e_1:  -1.7510526073242692
glc__D_e_1:  -1.5213179745960372
glc__D_e_1:  -2.138695426376672
glc__D_e_1:  -1.8642104563571966
glc__D_e_1:  -1.871771920149193
glc__D_e_1:  -3.716440660770499
glc__D_e_1:  -4.828685077318682
glc__D_e_1:  -3.7088889134390737
glc__D_e_1:  -4.421735202377032
glc__D_e_1:  -6.115897507282398
glc__D_e_1:  -4.875962260764068
glc__D_e_1:  -3.6291073580256983
glc__D_e_1:  -3.7614112017907306
glc__D_e_1:  -4.183555167578157
glc__D_e_1:  -6.9491763591497895
glc__D_e_1:  -3.8753096286759208
glc__D_e_1:  -2.463156267316701
glc__D_e_1:  -1.7964067195895739
glc__D_e_1:  -2.426956559297446
glc__D_e_1:  -1.5604418028743379
glc__D_e_1:  -1.3813216865281301
glc__D_e_1:  -0.6369912458712809
glc__D_e_1:  -1.1637315866757683
glc__D_e_1:  -2.1836247882893565
glc__D_e_1:  -0.339500031423951
glc__D_e_1:  -0.07571502036992672
strain_number:  0 4.266666666666667
strain_various:  0 2.528943565118755
strain_number:  1 0.0
strain_various:  1 0.0
fd_

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7,  2, 17,  9,  3,  2, 13, 19,  2, 19,  8, 17,  8,  7,  1,  2, 18,
        7, 10,  6, 16, 14, 10, 13, 13, 10, 15, 12,  4, 18]), 1: array([18,  6,  7,  2, 19, 19,  1,  3,  5,  3,  4,  9,  1, 16, 20, 19, 12,
        9, 15, 19,  7,  5, 20,  8,  1,  9, 17, 16, 11,  1])}
1
strain_number:  0 11.633333333333333
strain_various:  0 6.779790229465478
strain_number:  1 10.066666666666666
strain_various:  1 6.712840101046814
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7,  2, 17,  9,  3,  2, 13, 19,  2, 19,  8, 17,  8,  7,  1,  2, 18,
        7, 10,  6, 16, 14, 10, 13, 13, 10, 15, 12,  4, 18]), 1: array([18,  6,  7,  2, 19, 19,  1,  3,  5,  3,  4,  9,  1, 16, 20, 19, 12,
        9, 15, 19,  7,  5, 20,  8,  1,  9, 17, 16, 11,  1])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -33.59900180007606
strain_number:  0 45.36666666666667
strain_various:  0 14.11024056807285
strain_number:  1 1.8333333333333333
strain_various:  1 1.5293426329272617
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 864.3727802125635, 404.25927877591965, 317.9836577895789, 333.3079628704629, 377.389572927073, 392.6748073134297, 415.2970816465193, 428.6752434190129, 382.8472341205229, 395.035154748748, 363.0111896484203, 319.24379177486657, 321.7180968679729]
slip_r:  321.7180968679729
15
glc__D_e_1:  -0.7269768946433288
glc__D_e_1:  -11.295433593204134
glc__D_e_1:  -5.3446661702947935
glc__D_e_1:  -13.912651353444865
glc__D_e_1:  -4.023997714078064
glc__D_e_1:  -18.82228043688977
glc__D_e_1:  -1.7324693729217628
glc__D_e_1:  -11.603158530945452
glc__D_e_1:  -23.426010698352577
glc__D_e_1:  -2.4438335464252248
glc__D_e_1:  -28.184576891533148
glc__D_e_1:  -19.37265485412824
glc__D_e_1:  -0.9454956932952108
glc__D_e_1:  -1.8366813342517123
glc__D_e_1:  -24.98740777602173
glc__D_e_

glc__D_e_1:  -7.353356702468556
glc__D_e_1:  -10.959380981099592
glc__D_e_1:  -7.947263272040752
glc__D_e_1:  -7.4458661381160125
glc__D_e_1:  -8.131153838757886
glc__D_e_1:  -3.0699448154334
glc__D_e_1:  -3.1263004545500017
glc__D_e_1:  -4.646323691718928
glc__D_e_1:  -3.583467033139326
glc__D_e_1:  -1.5567794216483168
glc__D_e_1:  -4.7589031429625495
glc__D_e_1:  -2.6113224622091753
glc__D_e_1:  -3.8043828176228636
glc__D_e_1:  -3.930196565968129
glc__D_e_1:  -4.899880070079768
glc__D_e_1:  -0.8338366413154256
glc__D_e_1:  -1.421219942312986
glc__D_e_1:  -0.97604019195246
glc__D_e_1:  -1.276211802682477
glc__D_e_1:  -0.7665280362253819
glc__D_e_1:  -0.8915799176116708
glc__D_e_1:  -0.7307381059376343
glc__D_e_1:  -0.7071555804547016
glc__D_e_1:  -0.5628221532414254
glc__D_e_1:  -0.8620428894385652
glc__D_e_1:  -0.16770165697796602
glc__D_e_1:  -0.40378184617607227
glc__D_e_1:  -1.377294186233614
strain_number:  0 4.733333333333333
strain_various:  0 3.7499629627800624
strain_number: 

In [9]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 6,  1, 12, 17, 10, 15, 11, 17, 15, 14,  9,  7,  1,  8,  1, 14, 16,
       11,  4, 16, 18,  5,  1, 18,  5,  5, 18,  9, 17,  8]), 1: array([12,  8, 11, 11, 17, 17,  7,  1, 14, 18, 19, 10, 11,  1, 20, 12, 11,
        6,  1, 19, 17, 11, 17, 18, 15, 13,  9,  8, 16, 19])}
1
strain_number:  0 12.0
strain_various:  0 6.747839160304085
strain_number:  1 12.3
strain_various:  1 5.429241813243048
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 6,  1, 12, 17, 10, 15, 11, 17, 15, 14,  9,  7,  1,  8,  1, 14, 16,
       11,  4, 16, 18,  5,  1, 18,  5,  5, 18,  9, 17,  8]), 1: array([12,  8, 11, 11, 17, 17,  7,  1, 14, 18, 19, 10, 11,  1, 20, 12, 11,
        6,  1, 19, 17, 11, 17, 18, 15, 13,  9,  8, 16, 19])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404

glc__D_e_1:  -9.373698460682519
glc__D_e_1:  -21.12718351692596
glc__D_e_1:  -3.854373837692357
glc__D_e_1:  -1.463282502196732
glc__D_e_1:  -9.523954292178797
strain_number:  0 42.3
strain_various:  0 17.85898466692139
strain_number:  1 2.433333333333333
strain_various:  1 1.6868774571839993
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 911.3684039129083, 375.83269647019654, 403.34310593946816, 452.7430353444285, 529.3720067213488, 528.7087244744504, 529.5391861492483, 509.0270719334138, 505.1573541376979, 437.0484954794084, 408.92368145805636, 372.69504528115993, 366.9069074619488]
slip_r:  366.9069074619488
15
glc__D_e_1:  -0.03936980911437615
glc__D_e_1:  -3.9888417190599412
glc__D_e_1:  -17.517668795879953
glc__D_e_1:  -14.684839390704685
glc__D_e_1:  -20.356175726750397
glc__D_e_1:  -7.7854447125188235
glc__D_e_1:  -11.380920552806279
glc__D_e_1:  -9.733331432841432
glc__D_e_1:  -14.701902871752816
glc__D_e_1:  -0.16710979887913524
glc__D_e_1:  -13.568728559667925
glc__D_e_1:  -14.9

glc__D_e_1:  -4.38770375902766
glc__D_e_1:  -4.245626686218533
glc__D_e_1:  -2.752453019684072
glc__D_e_1:  -1.4128057869793809
glc__D_e_1:  -1.722044085515355
glc__D_e_1:  -0.4793817189675992
glc__D_e_1:  -0.016120268566923945
glc__D_e_1:  -0.8059453515767349
glc__D_e_1:  -0.4908003513551862
glc__D_e_1:  -0.4837294354191226
glc__D_e_1:  -3.0528971456392413
glc__D_e_1:  -6.87279731695802
glc__D_e_1:  -7.813461350468528
glc__D_e_1:  -8.473874777319017
glc__D_e_1:  -6.058631574600982
glc__D_e_1:  -2.665857729209505
glc__D_e_1:  -1.8399623037172135
glc__D_e_1:  -1.8747771449172825
glc__D_e_1:  -1.9356796336900626
glc__D_e_1:  -0.31797040580100433
glc__D_e_1:  -1.2528035855305006
glc__D_e_1:  -3.1368072097765998
glc__D_e_1:  -0.9710755305215781
glc__D_e_1:  -0.20883077108891934
glc__D_e_1:  -0.021653226784499324
glc__D_e_1:  -0.0006144097717281083
glc__D_e_1:  -0.9122013944220024
strain_number:  0 3.7333333333333334
strain_various:  0 3.032417444145109
strain_number:  1 0.0
strain_various:

In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 9,  9, 19, 13,  1,  2, 10,  2, 15,  5, 12,  6, 20, 11, 20,  3, 13,
       18, 10, 16, 11, 19, 20, 16,  7,  7, 10,  2, 17, 20]), 1: array([ 7,  8,  5, 18,  3, 13,  4,  5, 12, 20, 10, 14, 10, 13,  8,  5,  4,
       17,  5,  4, 18,  2,  7, 16, 14,  5,  5,  5,  4, 17])}
1
strain_number:  0 13.4
strain_various:  0 7.355723395198236
strain_number:  1 9.266666666666667
strain_various:  1 5.347481234708127
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 9,  9, 19, 13,  1,  2, 10,  2, 15,  5, 12,  6, 20, 11, 20,  3, 13,
       18, 10, 16, 11, 19, 20, 16,  7,  7, 10,  2, 17, 20]), 1: array([ 7,  8,  5, 18,  3, 13,  4,  5, 12, 20, 10, 14, 10, 13,  8,  5,  4,
       17,  5,  4, 18,  2,  7, 16, 14,  5,  5,  5,  4, 17])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -8.695612432488863
glc__D_e_1:  -21.102625684746826
glc__D_e_1:  -10.778991863541025
glc__D_e_1:  -19.67184764480744
glc__D_e_1:  -37.47551999507989
glc__D_e_1:  -45.10097026023919
glc__D_e_1:  -20.701481735178533
glc__D_e_1:  -31.119464342780418
glc__D_e_1:  -33.85014252486428
glc__D_e_1:  -20.04757697389179
strain_number:  0 41.2
strain_various:  0 18.334666618185345
strain_number:  1 0.16666666666666666
strain_various:  1 0.37267799624996495
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 706.9109378108991, 363.3825499062341, 393.79983612878345, 442.83112782096225, 432.93435291059126, 419.85443898226157, 366.3194684320148, 362.7279182235927, 397.49165428929734, 460.9374558556459, 387.5450904615502, 362.4557999575982, 350.53342077247]
slip_r:  350.53342077247
15
glc__D_e_1:  -8.281288391346415
glc__D_e_1:  -4.431682747768168
glc__D_e_1:  -17.331430157263192
glc__D_e_1:  -15.785147779066723
glc__D_e_1:  -17.683843650415348
glc__D_e_1:  -16.501567916253123
glc__D_e_1:  -12.8975

glc__D_e_1:  -0.954939264921072
glc__D_e_1:  -0.5019665512910036
glc__D_e_1:  -0.06334067731798732
glc__D_e_1:  -0.2744588951672926
glc__D_e_1:  -0.2783134759582795
glc__D_e_1:  -0.03193632683793923
glc__D_e_1:  -0.006387709602791336
glc__D_e_1:  -0.8874830810800555
glc__D_e_1:  -1.316079832695213
glc__D_e_1:  -1.9207031297714257
glc__D_e_1:  -1.5229832547177709
glc__D_e_1:  -0.8319227499320689
glc__D_e_1:  -1.8750647361852402
strain_number:  0 3.6
strain_various:  0 3.869539162915054
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 706.9109378108991, 363.3825499062341, 393.79983612878345, 442.83112782096225, 432.93435291059126, 419.85443898226157, 366.3194684320148, 362.7279182235927, 397.49165428929734, 460.9374558556459, 387.5450904615502, 362.4557999575982, 350.53342077247, 205.9272449504902, 192.97695113873615, 178.8830451429078, 187.23174824934858, 183.5810054585463, 257.45544574104946]
slip_r:  257.45544574104946
21
glc__D_e_1:  -1.64509404

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3,  6, 13, 18,  9, 19, 15,  7,  3, 14,  3, 16, 12,  1, 15, 19,  8,
       14,  5,  8,  2, 17, 20,  1, 15,  6,  5, 13, 15, 16]), 1: array([ 4,  6, 13, 10, 13,  6,  4,  5, 12, 20, 11, 19,  9,  4, 13,  6, 20,
       10,  7, 12, 11,  6, 17, 20, 13, 13,  6, 15, 12,  1])}
1
strain_number:  0 12.333333333333334
strain_various:  0 7.133644853010898
strain_number:  1 10.6
strain_various:  1 5.187163643713842
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3,  6, 13, 18,  9, 19, 15,  7,  3, 14,  3, 16, 12,  1, 15, 19,  8,
       14,  5,  8,  2, 17, 20,  1, 15,  6,  5, 13, 15, 16]), 1: array([ 4,  6, 13, 10, 13,  6,  4,  5, 12, 20, 11, 19,  9,  4, 13,  6, 20,
       10,  7, 12, 11,  6, 17, 20, 13, 13,  6, 15, 12,  1])}
_2:  {0: [0.6741983849766484, 0.2857391744693639], 1: [0.6741983849766484, 0.2857391744693639], 2: [0.6741983849766484, 0.2857391744693639], 3: [0.6741983849766484, 0.2857391744693639], 4: [0.6741983849766484, 0.2857391744693639], 5: [0.67419838

glc__D_e_1:  -1.1568662841152175
glc__D_e_1:  -17.665411726065088
glc__D_e_1:  -31.79591037045568
glc__D_e_1:  -2.3953256796041487
glc__D_e_1:  -16.518728567149505
glc__D_e_1:  -44.10569034675425
glc__D_e_1:  -1.1486029363399588
glc__D_e_1:  -0.5745494054031455
glc__D_e_1:  -13.53681769146534
strain_number:  0 43.53333333333333
strain_various:  0 15.78761821456577
strain_number:  1 1.9666666666666666
strain_various:  1 1.4255603186895396
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 911.917797695081, 331.51966403049994, 432.8474627153032, 437.4114690134427, 378.0068319838057, 403.4285429176559, 493.25437761960353, 437.27542556726, 401.4715865927927, 381.2048040845665, 379.46773045979705, 335.24609844520774, 334.4765414011133]
slip_r:  334.4765414011133
15
glc__D_e_1:  -16.023496709579657
glc__D_e_1:  -6.149544380671827
glc__D_e_1:  -6.6470349797948
glc__D_e_1:  -20.99729907296055
glc__D_e_1:  -28.559916702486053
glc__D_e_1:  -16.67458039788464
glc__D_e_1:  -21.324862715258966
glc__D_e_1: 

glc__D_e_1:  -2.3873740098388057
glc__D_e_1:  -0.7066341263528442
strain_number:  0 4.933333333333334
strain_various:  0 2.803965445975079
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 911.917797695081, 331.51966403049994, 432.8474627153032, 437.4114690134427, 378.0068319838057, 403.4285429176559, 493.25437761960353, 437.27542556726, 401.4715865927927, 381.2048040845665, 379.46773045979705, 335.24609844520774, 334.4765414011133, 339.0764434149378, 338.51407262174337, 311.96664666824756, 264.1053228598625, 227.21682561504542]
slip_r:  227.21682561504542
20
glc__D_e_1:  -4.95976712798175
glc__D_e_1:  -4.054169694349667
glc__D_e_1:  -6.1741602906212
glc__D_e_1:  -2.5852325073378815
glc__D_e_1:  -1.297321505820254
glc__D_e_1:  -1.0167156217046367
glc__D_e_1:  -1.6502631671771917
glc__D_e_1:  -0.6629549458583583
glc__D_e_1:  -1.4372263939974228
glc__D_e_1:  -0.4093558576961416
glc__D_e_1:  -0.2343968743867998
glc__D_e_1:  -1.0659001362701428
glc__D_

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([11, 12,  3, 18, 12,  4,  5, 15,  7, 17, 10, 13,  5,  3, 15,  6, 19,
        7, 16,  4, 14, 10,  9,  8, 10,  2,  1, 14, 20, 17]), 1: array([17,  6,  4, 15, 20,  7,  3, 15, 18,  7,  5, 17, 11,  7,  3, 12, 17,
       13, 12, 16,  3,  4, 11, 14, 13, 16, 18, 11, 14, 10])}
1
strain_number:  0 11.9
strain_various:  0 6.523547092392808
strain_number:  1 11.3
strain_various:  1 5.145548237716431
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([11, 12,  3, 18, 12,  4,  5, 15,  7, 17, 10, 13,  5,  3, 15,  6, 19,
        7, 16,  4, 14, 10,  9,  8, 10,  2,  1, 14, 20, 17]), 1: array([17,  6,  4, 15, 20,  7,  3, 15, 18,  7,  5, 17, 11,  7,  3, 12, 17,
       13, 12, 16,  3,  4, 11, 14, 13, 16, 18, 11, 14, 10])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404

glc__D_e_1:  -12.453321256566799
glc__D_e_1:  -31.296154123023236
glc__D_e_1:  -13.6054767207345
glc__D_e_1:  -21.21614210788165
glc__D_e_1:  -17.8640536371665
glc__D_e_1:  -0.9572411884640921
glc__D_e_1:  -29.87761170416906
glc__D_e_1:  -23.661854719031457
glc__D_e_1:  -0.673655421806572
glc__D_e_1:  -11.612634851329336
glc__D_e_1:  -20.830405362524683
glc__D_e_1:  -0.8828994895610371
glc__D_e_1:  -33.45464252199536
glc__D_e_1:  -31.443815921431124
glc__D_e_1:  -1.5225571119856554
glc__D_e_1:  -0.2673229869211937
glc__D_e_1:  -11.450075049686973
glc__D_e_1:  -27.397313786213168
glc__D_e_1:  -37.37648362802351
glc__D_e_1:  -23.908133369440197
glc__D_e_1:  -22.557981135982857
glc__D_e_1:  -24.082596061925134
glc__D_e_1:  -1.0943967647002495
glc__D_e_1:  -0.2793215678290312
glc__D_e_1:  -16.14519403977038
glc__D_e_1:  -28.059123053823946
glc__D_e_1:  -1.1785576686397157
glc__D_e_1:  -0.393461666449185
glc__D_e_1:  -20.942454547566108
glc__D_e_1:  -18.151571844405744
glc__D_e_1:  -36.4941

glc__D_e_1:  -2.7490394490285537
glc__D_e_1:  -1.578511259069468
glc__D_e_1:  -1.087298457877102
glc__D_e_1:  -2.961064228946724
glc__D_e_1:  -2.3762852905317486
glc__D_e_1:  -3.5634713969186667
glc__D_e_1:  -0.452411342228365
glc__D_e_1:  -2.4063804398216204
glc__D_e_1:  -2.180483386980802
glc__D_e_1:  -2.0957314644186082
glc__D_e_1:  -1.9958202041441049
glc__D_e_1:  -2.037570534970426
glc__D_e_1:  -0.8524528040289698
glc__D_e_1:  -1.535945402935989
glc__D_e_1:  -2.939707202319946
glc__D_e_1:  -1.6182600765987083
glc__D_e_1:  -3.1080913455790644
glc__D_e_1:  -1.7760214905784615
glc__D_e_1:  -4.134007014888398
glc__D_e_1:  -1.0303942556929666
glc__D_e_1:  -2.9584579687470645
glc__D_e_1:  -2.24779439442116
glc__D_e_1:  -2.2385090933348684
glc__D_e_1:  -1.0614431480142716
glc__D_e_1:  -1.1405586759972126
glc__D_e_1:  -0.025257656406988627
glc__D_e_1:  -0.0615599111058851
glc__D_e_1:  -0.04934959644315229
glc__D_e_1:  -0.39305751028982316
strain_number:  0 3.466666666666667
strain_various

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 5, 11, 18, 17, 16,  5,  6, 16, 15,  1,  5, 20,  5,  8, 20, 19, 10,
       10,  1, 15, 14,  5, 11, 16,  3,  9, 11, 20, 11, 11]), 1: array([ 8,  9,  3, 13,  2, 17, 18, 15, 14, 14,  2,  5, 18, 13,  2, 12, 16,
       18,  9,  5, 17,  2,  9, 10, 12,  6, 19,  3, 15,  3])}
1
strain_number:  0 13.133333333333333
strain_various:  0 6.878630354624063
strain_number:  1 10.3
strain_various:  1 5.7396283271073685
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 5, 11, 18, 17, 16,  5,  6, 16, 15,  1,  5, 20,  5,  8, 20, 19, 10,
       10,  1, 15, 14,  5, 11, 16,  3,  9, 11, 20, 11, 11]), 1: array([ 8,  9,  3, 13,  2, 17, 18, 15, 14, 14,  2,  5, 18, 13,  2, 12, 16,
       18,  9,  5, 17,  2,  9, 10, 12,  6, 19,  3, 15,  3])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49

glc__D_e_1:  -0.10648906353964449
glc__D_e_1:  -7.912961827076149
glc__D_e_1:  -24.61924161524913
glc__D_e_1:  -24.872245516730846
glc__D_e_1:  -8.061478624859554
glc__D_e_1:  -0.27674644894086065
glc__D_e_1:  -20.24372846922503
glc__D_e_1:  -21.263103237550382
glc__D_e_1:  -14.434241956145184
glc__D_e_1:  -26.05558766983409
glc__D_e_1:  -16.17081868827263
glc__D_e_1:  -19.484809335329995
glc__D_e_1:  -37.50877927282355
glc__D_e_1:  -0.8374595664230959
glc__D_e_1:  -1.1530147277022256
glc__D_e_1:  -17.018887199643576
strain_number:  0 42.333333333333336
strain_various:  0 14.60441333600528
strain_number:  1 1.2666666666666666
strain_various:  1 0.8137703743822469
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 776.0464551573297, 298.84414598559346, 326.5310128468023, 403.8411544595754, 367.7035672660673, 429.48788528722747, 460.2291519916133, 456.3209422029602, 378.2758118473869, 457.9075009414131, 462.48304721994276, 377.4311829223306, 394.0605941681765]
slip_r:  394.0605941681765
15
glc__

glc__D_e_1:  -0.6920347755800575
glc__D_e_1:  -0.7239222051709544
glc__D_e_1:  -1.1886252600000717
glc__D_e_1:  -0.5157947392815778
glc__D_e_1:  -0.057884334303309415
glc__D_e_1:  -1.6243496798267594
glc__D_e_1:  -1.2253147094683283
glc__D_e_1:  -2.0584869489246236
glc__D_e_1:  -1.954627693468945
glc__D_e_1:  -2.0593004598884423
glc__D_e_1:  -1.1223268941813758
glc__D_e_1:  -0.9904327414479963
glc__D_e_1:  -1.9206321125601398
glc__D_e_1:  -0.6961943729523756
glc__D_e_1:  -0.8760302033081389
glc__D_e_1:  -0.4614777313938633
glc__D_e_1:  -2.7544560512982073
glc__D_e_1:  -0.7540899108786626
glc__D_e_1:  -1.5262537727201688
glc__D_e_1:  -3.51290042510878
glc__D_e_1:  -1.9546455371635216
glc__D_e_1:  -2.11357633929964
glc__D_e_1:  -2.2177487193663277
glc__D_e_1:  -0.8771012692508946
glc__D_e_1:  -0.5350013974963626
glc__D_e_1:  -0.253711869941195
glc__D_e_1:  -0.6435691248958264
glc__D_e_1:  -1.2865374932210274
glc__D_e_1:  -0.6493517681418285
strain_number:  0 2.5
strain_various:  0 1.2041

In [9]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8,  2, 16, 15, 15, 17,  5,  1, 18,  4, 13, 12,  2, 13,  5, 16, 15,
       15, 14,  2, 10, 11,  4,  9, 15,  1,  5,  5, 14,  8]), 1: array([11, 18, 17,  2,  4,  6, 11,  8, 14, 19,  4,  1, 14,  9, 19, 11,  8,
        9, 14, 11,  4, 14, 16, 10, 13, 11, 14,  9, 17, 16])}
1
strain_number:  0 11.266666666666667
strain_various:  0 6.577402391690979
strain_number:  1 11.133333333333333
strain_various:  1 4.937835783237654
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8,  2, 16, 15, 15, 17,  5,  1, 18,  4, 13, 12,  2, 13,  5, 16, 15,
       15, 14,  2, 10, 11,  4,  9, 15,  1,  5,  5, 14,  8]), 1: array([11, 18, 17,  2,  4,  6, 11,  8, 14, 19,  4,  1, 14,  9, 19, 11,  8,
        9, 14, 11,  4, 14, 16, 10, 13, 11, 14,  9, 17, 16])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -0.15291613046138508
glc__D_e_1:  -25.555856347135304
glc__D_e_1:  -4.552263098765103
glc__D_e_1:  -22.462212993421396
glc__D_e_1:  -20.120235163617206
glc__D_e_1:  -17.61276222759033
glc__D_e_1:  -12.183913822125774
glc__D_e_1:  -17.501592756441056
glc__D_e_1:  -13.454252256611085
glc__D_e_1:  -1.1084799443005338
glc__D_e_1:  -5.872860561236443
glc__D_e_1:  -1.5399681508489032
glc__D_e_1:  -1.1739709203177076
glc__D_e_1:  -1.90898343800216
strain_number:  0 42.766666666666666
strain_various:  0 12.818432908207704
strain_number:  1 1.5
strain_various:  1 1.8929694486000912
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 860.9603850741622, 377.97979742136096, 407.4859411744087, 397.1588630667579, 372.2533006467217, 427.4370522021838, 437.03798759719814, 355.20368457319177, 387.5625396141864, 394.9243783576436, 353.8073245653035, 371.40864575632696, 333.4629990432511, 368.30541098529693]
slip_r:  368.30541098529693
16
glc__D_e_1:  -14.981417649760411
glc__D_e_1:  -0.4466245768867

glc__D_e_1:  -3.3300812032821394
glc__D_e_1:  -2.734564066569002
glc__D_e_1:  -2.1189224666693955
glc__D_e_1:  -1.0174379046709794
glc__D_e_1:  -1.9450657159912437
glc__D_e_1:  -3.9072812122723994
glc__D_e_1:  -7.095118174650564
glc__D_e_1:  -4.40450000053138
glc__D_e_1:  -7.776262905892931
glc__D_e_1:  -1.6368129714891209
glc__D_e_1:  -8.132435892368964
glc__D_e_1:  -1.2832537120941931
strain_number:  0 4.9
strain_various:  0 3.320140559273558
strain_number:  1 0.03333333333333333
strain_various:  1 0.17950549357115014
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 860.9603850741622, 377.97979742136096, 407.4859411744087, 397.1588630667579, 372.2533006467217, 427.4370522021838, 437.03798759719814, 355.20368457319177, 387.5625396141864, 394.9243783576436, 353.8073245653035, 371.40864575632696, 333.4629990432511, 368.30541098529693, 319.5599429387467, 216.23944448381278, 261.0247861003613, 231.4633949152872, 199.7709627329193]
slip_r:  199.7709627329193
21
glc__D_e_1:  -0.40053043702055846


In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7,  1, 12,  1,  2, 10,  1, 13, 12, 16,  6, 18, 12, 12, 18, 12,  2,
        9, 19,  4,  2, 18, 12,  9, 15,  7, 13,  6,  6, 16]), 1: array([12, 15, 18, 15, 17,  5, 10,  1,  5, 19, 14,  7,  8, 17, 10, 15,  6,
        3, 11,  8, 10, 17,  8,  9,  9,  7, 18,  3,  7,  9])}
1
strain_number:  0 11.233333333333333
strain_various:  0 6.691702988693452
strain_number:  1 10.433333333333334
strain_various:  1 4.944244690097321
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7,  1, 12,  1,  2, 10,  1, 13, 12, 16,  6, 18, 12, 12, 18, 12,  2,
        9, 19,  4,  2, 18, 12,  9, 15,  7, 13,  6,  6, 16]), 1: array([12, 15, 18, 15, 17,  5, 10,  1,  5, 19, 14,  7,  8, 17, 10, 15,  6,
        3, 11,  8, 10, 17,  8,  9,  9,  7, 18,  3,  7,  9])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -9.510049658345746
glc__D_e_1:  -19.818092411714474
glc__D_e_1:  -8.023787704733122
glc__D_e_1:  -8.044744727606513
glc__D_e_1:  -25.184846386394796
glc__D_e_1:  -25.070497036874297
glc__D_e_1:  -24.925492543340933
glc__D_e_1:  -13.477647064192848
glc__D_e_1:  -12.465448195576823
glc__D_e_1:  -19.35655728046392
glc__D_e_1:  -15.792982930960918
glc__D_e_1:  -12.373035033012886
glc__D_e_1:  -24.424971237795106
glc__D_e_1:  -26.346672260715874
glc__D_e_1:  -29.69721327336226
glc__D_e_1:  -16.42519502900804
glc__D_e_1:  -31.974654273779258
glc__D_e_1:  -0.16096264911084068
glc__D_e_1:  -16.02683512105219
glc__D_e_1:  -23.262518283295137
glc__D_e_1:  -28.418822825550826
strain_number:  0 37.13333333333333
strain_various:  0 17.775888788531
strain_number:  1 0.9333333333333333
strain_various:  1 1.5691469727919758
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 766.4025169876561, 385.97615200846633, 401.9927523456936, 465.97548175508706, 453.1952503636714, 397.2772603975915, 444.5090

glc__D_e_1:  -11.253262932856114
glc__D_e_1:  -9.179355803740435
glc__D_e_1:  -16.694814948025872
glc__D_e_1:  -3.0528308072658894
glc__D_e_1:  -2.0308980514062114
glc__D_e_1:  -2.254034471962535
glc__D_e_1:  -1.1660794599975657
glc__D_e_1:  -1.3115999284942939
glc__D_e_1:  -0.339429648633804
glc__D_e_1:  -0.052792575554533316
glc__D_e_1:  -0.21925597639108707
glc__D_e_1:  -1.7065103942701068
glc__D_e_1:  -0.2091346678814371
glc__D_e_1:  -2.347963023501707
glc__D_e_1:  -1.5629991049651963
glc__D_e_1:  -1.6293179635671773
glc__D_e_1:  -1.9441340370741713
glc__D_e_1:  -0.009909056376773528
glc__D_e_1:  -0.8924688663401594
glc__D_e_1:  -0.9574587697685264
glc__D_e_1:  -0.17924276721423138
glc__D_e_1:  -0.578912830095144
glc__D_e_1:  -0.05191066189054672
strain_number:  0 4.733333333333333
strain_various:  0 7.928569998569871
strain_number:  1 0.06666666666666667
strain_various:  1 0.3590109871423002
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 766.4025169876561, 385.97615200846633, 401.9927

In [11]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([19,  1, 17,  7,  5,  2, 16, 18, 11, 18, 17,  7, 10, 19, 18, 12, 17,
       11,  6, 14,  1, 12,  2,  3,  4,  8, 18,  2, 19,  8]), 1: array([ 5,  3, 13,  7,  4, 19, 16, 13,  6,  1,  7, 16, 19, 19,  6, 14, 18,
       13, 14, 15,  3, 19, 14, 17, 16,  4,  4, 14, 18,  6])}
1
strain_number:  0 12.433333333333334
strain_various:  0 7.490809183407506
strain_number:  1 11.433333333333334
strain_various:  1 5.891707920647646
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([19,  1, 17,  7,  5,  2, 16, 18, 11, 18, 17,  7, 10, 19, 18, 12, 17,
       11,  6, 14,  1, 12,  2,  3,  4,  8, 18,  2, 19,  8]), 1: array([ 5,  3, 13,  7,  4, 19, 16, 13,  6,  1,  7, 16, 19, 19,  6, 14, 18,
       13, 14, 15,  3, 19, 14, 17, 16,  4,  4, 14, 18,  6])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -0.06803993863747326
glc__D_e_1:  -10.210098577142015
glc__D_e_1:  -22.139484197218707
glc__D_e_1:  -0.002957563691930787
glc__D_e_1:  -6.623747880336693
glc__D_e_1:  -14.949138834857898
glc__D_e_1:  -1.0830198104167277
glc__D_e_1:  -0.9261292377265095
glc__D_e_1:  -9.507147739839194
glc__D_e_1:  -13.499977216428768
glc__D_e_1:  -1.1542049041182167
glc__D_e_1:  -0.8526163775441944
glc__D_e_1:  -6.831901319003784
glc__D_e_1:  -12.966758486403322
glc__D_e_1:  -0.6209861740927707
glc__D_e_1:  -7.327257580871078
glc__D_e_1:  -20.21984321442105
glc__D_e_1:  -0.9410113898899155
glc__D_e_1:  -1.3833202811244485
glc__D_e_1:  -15.167805904543322
glc__D_e_1:  -17.163015892797908
glc__D_e_1:  -0.256203444095501
glc__D_e_1:  -12.95257712517364
strain_number:  0 41.06666666666667
strain_various:  0 21.792863255866333
strain_number:  1 2.1
strain_various:  1 2.5079872407968904
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 903.6430816345675, 358.9649254254517, 409.69866069173355, 451.980662

glc__D_e_1:  -1.1687412242202406
glc__D_e_1:  -1.4341291701562637
glc__D_e_1:  -2.5927694451469643
glc__D_e_1:  -5.293261624011119
glc__D_e_1:  -4.819040637357189
glc__D_e_1:  -10.337709708364779
glc__D_e_1:  -0.3639567718829557
glc__D_e_1:  -5.974642914954412
glc__D_e_1:  -13.705464462159252
glc__D_e_1:  -4.9758103856232685
glc__D_e_1:  -11.131866583423676
glc__D_e_1:  -0.4934425472225801
glc__D_e_1:  -3.8241064576340627
glc__D_e_1:  -2.7481026704573073
strain_number:  0 5.733333333333333
strain_various:  0 5.303667494186347
strain_number:  1 0.06666666666666667
strain_various:  1 0.35901098714230023
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 903.6430816345675, 358.9649254254517, 409.69866069173355, 451.9806626884551, 440.659380896881, 429.08879567118856, 462.90024037720855, 448.73102862050234, 435.9345591036099, 412.76782786162914, 321.15349372240127, 375.4900121607127, 430.9142385355241, 326.14208063953873, 356.68184655769585, 271.644315008946, 255.87510519862295, 239.51857024554394

In [12]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8,  9, 14,  5,  9,  4,  1,  6,  8,  4, 16, 11,  2, 15, 16,  8,  4,
       16, 11, 18,  8,  5, 10, 17,  2, 16, 15, 18, 17, 11]), 1: array([ 4,  9, 16,  4,  6,  5, 16, 12, 11, 17, 12, 18, 13, 13, 20, 20, 17,
       16, 16, 10,  7, 11, 14,  7,  4, 15,  9,  3, 16,  2])}
1
strain_number:  0 11.766666666666667
strain_various:  0 6.401475524352874
strain_number:  1 11.433333333333334
strain_various:  1 5.264239947249956
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8,  9, 14,  5,  9,  4,  1,  6,  8,  4, 16, 11,  2, 15, 16,  8,  4,
       16, 11, 18,  8,  5, 10, 17,  2, 16, 15, 18, 17, 11]), 1: array([ 4,  9, 16,  4,  6,  5, 16, 12, 11, 17, 12, 18, 13, 13, 20, 20, 17,
       16, 16, 10,  7, 11, 14,  7,  4, 15,  9,  3, 16,  2])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -18.801087763565608
glc__D_e_1:  -14.355552143185578
glc__D_e_1:  -21.354844679739468
glc__D_e_1:  -14.888021386835998
strain_number:  0 33.43333333333333
strain_various:  0 10.597746091609396
strain_number:  1 2.033333333333333
strain_various:  1 2.3732303348436754
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 622.8784249161571, 329.8325595972655, 338.34728066051593, 385.72341767598556, 399.06044635189375, 467.04182608729195, 427.2949892033853, 433.2085782757847, 424.64809861292997, 420.78549720039996, 373.58451995542305, 439.04426097357543, 458.9226096749657]
slip_r:  458.9226096749657
15
glc__D_e_1:  -1.6634331880160467
glc__D_e_1:  -13.337762127600865
glc__D_e_1:  -2.329337912155335
glc__D_e_1:  -1.0895276896555712
glc__D_e_1:  -10.711239616029493
glc__D_e_1:  -11.821482308879403
glc__D_e_1:  -1.6647307571319825
glc__D_e_1:  -1.5057644480749928
glc__D_e_1:  -7.4850493895345815
glc__D_e_1:  -18.133840673911692
glc__D_e_1:  -2.564376322074306
glc__D_e_1:  -2.291638902637518

glc__D_e_1:  -0.4993450464851861
glc__D_e_1:  -1.8641001431104633
glc__D_e_1:  -0.39766494289679777
glc__D_e_1:  -0.6356766657868279
glc__D_e_1:  -0.2845908892758161
strain_number:  0 8.566666666666666
strain_various:  0 11.43002867693496
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 622.8784249161571, 329.8325595972655, 338.34728066051593, 385.72341767598556, 399.06044635189375, 467.04182608729195, 427.2949892033853, 433.2085782757847, 424.64809861292997, 420.78549720039996, 373.58451995542305, 439.04426097357543, 458.9226096749657, 362.22262368962856, 378.1839463046429, 294.9934316690334, 243.9742624396774, 234.31632741647175, 196.7051713826425]
slip_r:  196.7051713826425
21
glc__D_e_1:  -10.162250488072859
glc__D_e_1:  -4.513511899240103
glc__D_e_1:  -12.049000153619716
glc__D_e_1:  -8.044432488580195
glc__D_e_1:  -23.246530541062896
glc__D_e_1:  -7.543699121966423
glc__D_e_1:  -15.809481565873716
glc__D_e_1:  -12.777583324888512
glc__D_e_1:

strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 622.8784249161571, 329.8325595972655, 338.34728066051593, 385.72341767598556, 399.06044635189375, 467.04182608729195, 427.2949892033853, 433.2085782757847, 424.64809861292997, 420.78549720039996, 373.58451995542305, 439.04426097357543, 458.9226096749657, 362.22262368962856, 378.1839463046429, 294.9934316690334, 243.9742624396774, 234.31632741647175, 196.7051713826425, 237.34845417740155, 219.57461166284696, 145.39204545454544, 66.9047619047619, 101.07142857142857, 67.5, 67.5, 15.0, 15.0, 0.0]
slip_r:  0.0
The initial distribution of microorganism is:  {}
The change process of microorganism distribution is:  {1: {0: [4.489717762411146, 4.98857529156794, 7.981720466508705, 2.9931451749407643, 4.98857529156794, 1.9954301166271762, 0.49885752915679404, 3.4920027040975583, 4.489717762411146, 1.9954301166271762, 9.478293053979087, 6.485147879038323, 0.997715058